In [25]:
import pandas as pd
import time
import joblib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, chi2

In [2]:
df = pd.read_pickle(r'D:\JupyterNotebook\DataProcess\preprocess_SWnltk_stemming.pkl')
dfNo = pd.read_pickle(r'D:\JupyterNotebook\DataProcess\preprocess_SWnltk_nostemming.pkl')

In [12]:
kf = KFold(n_splits=5)
i = 1
for train_index, test_index in kf.split(df):
#     print(test_index)
#     print(train_index[0],"-",train_index[len(train_index)-1])
    print(test_index[0],"+",test_index[len(test_index)-1])
    X_train = df['text'].iloc[train_index[0]:train_index[len(train_index)-1]]
    X_test = df['text'].iloc[test_index[0]:test_index[len(test_index)-1]]
    y_train = df['label'].iloc[train_index[0]:train_index[len(train_index)-1]].astype('int')
    y_test = df['label'].iloc[test_index[0]:test_index[len(test_index)-1]].astype('int')
    
    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    modelknn2 = KNeighborsClassifier(n_neighbors=13, weights='uniform', metric='euclidean')
    modelknn2.fit(X_train,y_train)
    y_pred = modelknn2.predict(X_test)
    print(classification_report(y_test,y_pred))
    result = modelknn2.score(X_test, y_test)
    print("Fold ke-",i,"=",result)
    print("----------------")
    i+=1

0 + 71
              precision    recall  f1-score   support

           0       0.60      0.50      0.55         6
           1       0.67      0.67      0.67         6
           2       0.86      1.00      0.92         6
           3       0.62      0.83      0.71         6
           4       0.62      0.83      0.71         6
           5       0.80      0.67      0.73         6
           6       0.71      0.83      0.77         6
           7       0.56      0.83      0.67         6
           8       0.60      0.50      0.55         6
           9       1.00      0.17      0.29         6
          10       1.00      0.67      0.80         6
          11       0.67      0.80      0.73         5

    accuracy                           0.69        71
   macro avg       0.73      0.69      0.67        71
weighted avg       0.73      0.69      0.67        71

Fold ke- 1 = 0.6901408450704225
----------------
72 + 143
              precision    recall  f1-score   support

           0 

In [15]:
def split(strt,fnsh,df):
    X_test = df['text'].iloc[strt:fnsh]
    y_test = df['label'].iloc[strt:fnsh].astype('int')
    df.drop(df.index[strt:fnsh])
    X_train = df['text']
    y_train = df['label'].astype('int')    
    return X_train, X_test, y_train, y_test

In [16]:
def select_features(train_X, train_y, test_X):
#     total fitur 7343
    selector = SelectKBest(chi2, k=5000)
    selector.fit(train_X, train_y)
    train_X = selector.transform(train_X)
    test_X = selector.transform(test_X)
    return train_X, test_X

In [17]:
def classifier(df,distance_name,k_neighbor):
    print("Nama distance=",distance_name)
    print("Jumlah K tetangga=",k_neighbor)
    spl = [0,36,72,108,144,180,216,252,288,324,360]
    tempspl = []
    i = 0
    akurasimax=0
    while (i<len(spl)-1):
        if len(spl)-2 != len(tempspl):
            X_train, X_test, y_train, y_test = split(spl[i],spl[i+2],df)
            tempspl.append(spl[i])
    #   ketika array[i+2] == null (looping terakhir)
        elif (len(spl)-len(tempspl)) == 2:
    #       split last and first
            X_test1 = df['text'].iloc[spl[i]:spl[i+1]]
            y_test1 = df['label'].iloc[spl[i]:spl[i+1]].astype('int')
            X_test2 = df['text'].iloc[tempspl[0]:tempspl[1]]
            y_test2 = df['label'].iloc[tempspl[0]:tempspl[1]].astype('int')
    #       concat last and first
            X_test = pd.concat([X_test1, X_test2], axis=0)
            y_test = pd.concat([y_test1, y_test2], axis=0)
    #       split except last and first
            X_train = df['text'].iloc[tempspl[1]:tempspl[len(tempspl)-1]]
            y_train = df['label'].iloc[tempspl[1]:tempspl[len(tempspl)-1]].astype('int')
        
        vectorizer = TfidfVectorizer()
        X_train = vectorizer.fit_transform(X_train)
        X_test = vectorizer.transform(X_test)
        
        X_train,X_test = select_features(X_train, y_train, X_test)

        modelknn = KNeighborsClassifier(n_neighbors=k_neighbor, weights='uniform', metric=distance_name)
        modelknn.fit(X_train,y_train)
        y_pred = modelknn.predict(X_test)
        print(classification_report(y_test,y_pred))
        result = modelknn.score(X_test, y_test)
        print("Fold ke-",i+1,"=",result)
        if (result >= akurasimax):
            akurasimax = result
            foldke = i+1
        i+=1
    print("Akurasi terbaik adalah",akurasimax,"pada fold ke-",foldke)
    print("------------------------------------")
    return akurasimax

In [18]:
%%time
# %%capture cap --no-stderr
start_time = time.time()
dfcount = [df,dfNo]
distancecount = ['euclidean','manhattan','minkowski']
kcount = [1,3,5,7,9,11,13,15]
for x in range(2):
    arrd1 = []
    arrd2 = []
    arrd3 = []
    for y in range(len(distancecount)):
        maxresult = []
        for z in range(len(kcount)):
            if x==0:
                print('Dengan Stemming')
            else:
                print('Tanpa Stemming')
            tempmax = classifier(dfcount[x],distancecount[y],kcount[z])
            maxresult.append(tempmax)
            if y==0:
                arrd1.append(tempmax)
            elif y==1:
                arrd2.append(tempmax)
            elif y==2:
                arrd3.append(tempmax)
#         tapi fold yang dipakai beda beda
    plt.plot(kcount, arrd1, label = "euclidean")
    plt.plot(kcount, arrd2, label = "manhattan")
    plt.plot(kcount, arrd3, label = "minkowski")
    plt.xticks(kcount)
    if x==0:
        plt.title("Stemming")
    else:
        plt.title("Tidak Stemming")
    plt.xlabel('Fold')
    plt.ylabel('Akurasi')
    plt.legend()
    plt.pause(0.05)
plt.show
print("Lama Proses %s seconds" % (time.time() - start_time))
# with open('5000_fitur_SWnltkv2.txt', 'w') as f:
#     f.write(cap.stdout)

Dengan Stemming
Nama distance= euclidean
Jumlah K tetangga= 1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         6
           3       1.00      1.00      1.00         6
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00         6
           9       1.00      1.00      1.00         6
          10       1.00      1.00      1.00         6
          11       1.00      1.00      1.00         6

    accuracy                           1.00        72
   macro avg       1.00      1.00      1.00        72
weighted avg       1.00      1.00      1.00        72

Fold ke- 1 = 1.0
              precision    recall  f1-score   support


              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       0.86      1.00      0.92         6
           2       0.86      1.00      0.92         6
           3       0.83      0.83      0.83         6
           4       0.75      1.00      0.86         6
           5       1.00      0.83      0.91         6
           6       1.00      0.83      0.91         6
           7       1.00      0.67      0.80         6
           8       0.67      1.00      0.80         6
           9       1.00      1.00      1.00         6
          10       1.00      0.50      0.67         6
          11       1.00      0.83      0.91         6

    accuracy                           0.88        72
   macro avg       0.90      0.88      0.87        72
weighted avg       0.90      0.88      0.87        72

Fold ke- 1 = 0.875
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
     

Fold ke- 1 = 0.75
              precision    recall  f1-score   support

           0       0.83      0.83      0.83         6
           1       0.80      0.67      0.73         6
           2       0.83      0.83      0.83         6
           3       0.75      1.00      0.86         6
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         6
           6       1.00      0.67      0.80         6
           7       0.67      1.00      0.80         6
           8       0.80      0.67      0.73         6
           9       0.83      0.83      0.83         6
          10       1.00      0.67      0.80         6
          11       0.86      1.00      0.92         6

    accuracy                           0.85        72
   macro avg       0.86      0.85      0.84        72
weighted avg       0.86      0.85      0.84        72

Fold ke- 2 = 0.8472222222222222
              precision    recall  f1-score   support

           0       0.56     

              precision    recall  f1-score   support

           0       0.62      0.83      0.71         6
           1       0.80      0.67      0.73         6
           2       0.86      1.00      0.92         6
           3       0.67      0.67      0.67         6
           4       0.75      1.00      0.86         6
           5       1.00      0.83      0.91         6
           6       1.00      0.67      0.80         6
           7       0.57      0.67      0.62         6
           8       0.62      0.83      0.71         6
           9       0.83      0.83      0.83         6
          10       1.00      0.50      0.67         6
          11       1.00      0.83      0.91         6

    accuracy                           0.78        72
   macro avg       0.81      0.78      0.78        72
weighted avg       0.81      0.78      0.78        72

Fold ke- 1 = 0.7777777777777778
              precision    recall  f1-score   support

           0       0.80      0.67      0.73   

              precision    recall  f1-score   support

           0       0.75      1.00      0.86         6
           1       0.75      1.00      0.86         6
           2       1.00      0.83      0.91         6
           3       0.83      0.83      0.83         6
           4       1.00      1.00      1.00         6
           5       1.00      0.83      0.91         6
           6       1.00      0.83      0.91         6
           7       1.00      1.00      1.00         6
           8       0.62      0.83      0.71         6
           9       0.60      0.50      0.55         6
          10       1.00      0.67      0.80         6
          11       1.00      1.00      1.00         6

    accuracy                           0.86        72
   macro avg       0.88      0.86      0.86        72
weighted avg       0.88      0.86      0.86        72

Fold ke- 2 = 0.8611111111111112
              precision    recall  f1-score   support

           0       0.71      0.83      0.77   

...

Fold ke- 10 = 0.5416666666666666
Akurasi terbaik adalah 0.8472222222222222 pada fold ke- 3
------------------------------------
Dengan Stemming
Nama distance= minkowski
Jumlah K tetangga= 9
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       0.86      1.00      0.92         6
           2       1.00      1.00      1.00         6
           3       0.83      0.83      0.83         6
           4       0.62      0.83      0.71         6
           5       1.00      0.67      0.80         6
           6       1.00      0.67      0.80         6
           7       1.00      0.67      0.80         6
           8       0.55      1.00      0.71         6
           9       0.67      0.67      0.67         6
          10       1.00      0.67      0.80         6
          11       1.00      0.83      0.91         6

    accuracy                           0.82        72
   macro avg       0.87      0.82      0.82        7

              precision    recall  f1-score   support

           0       0.71      0.83      0.77         6
           1       0.67      1.00      0.80         6
           2       1.00      0.83      0.91         6
           3       0.83      0.83      0.83         6
           4       0.71      0.83      0.77         6
           5       1.00      0.83      0.91         6
           6       1.00      0.67      0.80         6
           7       1.00      0.67      0.80         6
           8       0.60      1.00      0.75         6
           9       0.83      0.83      0.83         6
          10       1.00      0.33      0.50         6
          11       0.86      1.00      0.92         6

    accuracy                           0.81        72
   macro avg       0.85      0.81      0.80        72
weighted avg       0.85      0.81      0.80        72

Fold ke- 1 = 0.8055555555555556
              precision    recall  f1-score   support

           0       0.56      0.83      0.67   

Fold ke- 1 = 0.7916666666666666
              precision    recall  f1-score   support

           0       0.56      0.83      0.67         6
           1       0.83      0.83      0.83         6
           2       1.00      0.67      0.80         6
           3       0.86      1.00      0.92         6
           4       0.67      1.00      0.80         6
           5       1.00      0.67      0.80         6
           6       1.00      0.83      0.91         6
           7       1.00      1.00      1.00         6
           8       0.62      0.83      0.71         6
           9       0.75      0.50      0.60         6
          10       1.00      0.67      0.80         6
          11       1.00      1.00      1.00         6

    accuracy                           0.82        72
   macro avg       0.86      0.82      0.82        72
weighted avg       0.86      0.82      0.82        72

Fold ke- 2 = 0.8194444444444444
              precision    recall  f1-score   support

           0  

              precision    recall  f1-score   support

           0       0.42      0.83      0.56         6
           1       0.83      0.83      0.83         6
           2       1.00      0.50      0.67         6
           3       0.80      0.67      0.73         6
           4       0.62      0.83      0.71         6
           5       0.83      0.83      0.83         6
           6       1.00      0.83      0.91         6
           7       0.83      0.83      0.83         6
           8       0.83      0.83      0.83         6
           9       1.00      0.67      0.80         6
          10       1.00      0.67      0.80         6
          11       0.86      1.00      0.92         6

    accuracy                           0.78        72
   macro avg       0.84      0.78      0.79        72
weighted avg       0.84      0.78      0.79        72

Fold ke- 1 = 0.7777777777777778
              precision    recall  f1-score   support

           0       0.45      0.83      0.59   

KeyboardInterrupt: 

# try--

In [3]:
X_train = df['text'].iloc[72:]
X_test = df['text'].iloc[:72]
y_train = df['label'].iloc[72:].astype('int')
y_test = df['label'].iloc[:72].astype('int')

In [ ]:
X = df['text']
y = df['label'].astype('int')
# X = dfNo['text']
# y = dfNo['label'].astype('int')

In [4]:
# Pembentukan tf-idf untuk pembobotan
vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(X)

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train.shape)

In [ ]:
print(vectorizer.vocabulary_)
# print(len(vectorizer.get_feature_names()))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

In [5]:
%%time
modelknn2 = KNeighborsClassifier(n_neighbors=13, weights='uniform', metric='euclidean')
modelknn2.fit(X_train,y_train)

Wall time: 2.02 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=13, p=2,
                     weights='uniform')

In [ ]:
filename = 'D:\JupyterNotebook\Model\model_nostemming_173_splitmanual.sav'
joblib.dump(modelknn2, filename)

In [ ]:
filename = 'D:\JupyterNotebook\Model\model_stemming.sav'
loaded_model = joblib.load(filename)
# result = loaded_model.score(X_test, y_test)
# print(result)

In [6]:
y_pred = modelknn2.predict(X_test)
y_pred

array([ 0,  1,  2,  3,  4,  5,  6,  0,  7,  4, 10,  4,  2,  1,  2,  8,  4,
        5,  6,  7,  8,  6,  1, 11,  0,  1,  2,  3,  5,  4,  6,  7, 11,  9,
        1, 11, 11,  1,  2,  3,  4,  5,  3,  7,  8,  7, 10, 11,  7,  7,  2,
        3,  4,  0,  6,  7,  3,  8, 10, 11,  0, 11,  2,  3,  4,  5,  6,  7,
        8,  6, 10, 11])

In [ ]:
y_pred = loaded_model.predict(X_test)
y_pred

In [7]:
modelknn2.predict_proba(X_test)

array([[0.30769231, 0.        , 0.        , 0.07692308, 0.23076923,
        0.        , 0.        , 0.23076923, 0.        , 0.07692308,
        0.        , 0.07692308],
       [0.        , 0.46153846, 0.07692308, 0.07692308, 0.        ,
        0.15384615, 0.        , 0.        , 0.15384615, 0.07692308,
        0.        , 0.        ],
       [0.        , 0.        , 0.76923077, 0.        , 0.        ,
        0.23076923, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.23076923, 0.07692308, 0.        , 0.46153846, 0.        ,
        0.        , 0.07692308, 0.        , 0.        , 0.07692308,
        0.07692308, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.53846154,
        0.30769231, 0.        , 0.        , 0.15384615, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.15384615, 0.        , 0.        ,
        0.46153846, 0.        , 0.        , 0.        , 0.07692308,
        0.23076923,

In [8]:
print(confusion_matrix(y_test, y_pred))

[[3 0 1 0 0 0 0 1 0 0 0 1]
 [0 4 0 0 0 0 0 1 0 0 0 1]
 [0 0 6 0 0 0 0 0 0 0 0 0]
 [0 0 0 5 0 0 0 0 1 0 0 0]
 [0 0 0 0 5 1 0 0 0 0 0 0]
 [1 0 0 0 1 4 0 0 0 0 0 0]
 [0 0 0 1 0 0 5 0 0 0 0 0]
 [1 0 0 0 0 0 0 5 0 0 0 0]
 [0 0 0 1 0 0 0 1 3 0 0 1]
 [0 0 0 0 1 0 2 1 1 1 0 0]
 [0 2 0 0 0 0 0 0 0 0 4 0]
 [0 0 0 0 1 0 0 0 0 0 0 5]]


In [9]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.60      0.50      0.55         6
           1       0.67      0.67      0.67         6
           2       0.86      1.00      0.92         6
           3       0.71      0.83      0.77         6
           4       0.62      0.83      0.71         6
           5       0.80      0.67      0.73         6
           6       0.71      0.83      0.77         6
           7       0.56      0.83      0.67         6
           8       0.60      0.50      0.55         6
           9       1.00      0.17      0.29         6
          10       1.00      0.67      0.80         6
          11       0.62      0.83      0.71         6

    accuracy                           0.69        72
   macro avg       0.73      0.69      0.68        72
weighted avg       0.73      0.69      0.68        72



In [10]:
result = modelknn2.score(X_test, y_test)
print(result)

0.6944444444444444


In [11]:
result_class = pd.DataFrame({'Actual Result':y_test, 'Predicted Result':y_pred})
result_class

,Actual Result,Predicted Result
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
...,...,...
67,7,7
68,8,8
69,9,6
70,10,10


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(result_class)